In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanSquaredError
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import get_file



# Download and extract DIV2K dataset
dataset_url = "http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip"
dataset_path = get_file("DIV2K_train_HR.zip", dataset_url, extract=True, cache_dir="/content/")
dataset_folder = "/content/DIV2K_train_HR"

3530603713/3530603713 [==============================] - 145s 0us/step


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, MaxPooling2D, UpSampling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Set the path to the downloaded DIV2K dataset folder
dataset_folder = "/content/datasets/DIV2K_train_HR"

# Load and preprocess the DIV2K dataset
image_list = os.listdir(dataset_folder)
x_train, x_val = train_test_split(image_list, test_size=0.2, random_state=42)

def preprocess_image(image_path):
    img = load_img(image_path)
    img = img_to_array(img)
    img = img / 255.0
    return img

x_train = np.array([preprocess_image(os.path.join(dataset_folder, img_path)) for img_path in x_train])
x_val = np.array([preprocess_image(os.path.join(dataset_folder, img_path)) for img_path in x_val])

# Define the autoencoder model
input_img = Input(shape=(None, None, 3))

# Encoder layers
x = Conv2D(128, (3, 3), activation='relu', padding='same')(input_img)
x = Dropout(0.2)(x)  # Add dropout
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Dropout(0.2)(x)  # Add dropout
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

# Decoder layers
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Dropout(0.2)(x)  # Add dropout
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = Dropout(0.2)(x)  # Add dropout
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)

decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Set the learning rate and decay rate
lr = 0.001
decay_rate = lr / 80

# Create the model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=Adam(lr=lr, decay=decay_rate), loss='mse')

# Train the model
autoencoder.fit(x_train, x_train, epochs=10, batch_size=16, validation_data=(x_val, x_val))

# Evaluate the model
decoded_imgs = autoencoder.predict(x_val)



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt

# Set the path to the downloaded DIV2K dataset folder
dataset_folder = "/content/datasets/DIV2K_train_HR"  # Replace with the actual path

# Set the desired image size
image_size = (256, 256)  # Replace with your desired image size

# Load and preprocess the dataset
def load_dataset():
    images = []
    img_list = os.listdir(dataset_folder)
    for img_name in img_list:
        img_path = os.path.join(dataset_folder, img_name)
        img = load_img(img_path, target_size=image_size)
        img = img_to_array(img) / 255.0  # Normalize pixel values
        images.append(img)
    images = np.array(images)
    return images

# Split dataset into training and testing sets
images = load_dataset()
train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

# Define the compression model
input_img = Input(shape=image_size + (3,))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded_img = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Create the model
autoencoder = Model(input_img, decoded_img)
autoencoder.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model
autoencoder.fit(train_images, train_images, epochs=10, batch_size=16, validation_data=(test_images, test_images))

# Reconstruct images
reconstructed_images = autoencoder.predict(test_images)

Epoch 1/10
40/40 [==============================] - 382s 10s/step - loss: 0.0671 - val_loss: 0.0424
Epoch 2/10
40/40 [==============================] - 362s 9s/step - loss: 0.0367 - val_loss: 0.0295
Epoch 3/10
40/40 [==============================] - 384s 10s/step - loss: 0.0274 - val_loss: 0.0246
Epoch 4/10
40/40 [==============================] - 376s 9s/step - loss: 0.0242 - val_loss: 0.0218
Epoch 5/10
40/40 [==============================] - 349s 9s/step - loss: 0.0225 - val_loss: 0.0213
Epoch 6/10
40/40 [==============================] - 340s 9s/step - loss: 0.0219 - val_loss: 0.0204
Epoch 7/10
40/40 [==============================] - 349s 9s/step - loss: 0.0212 - val_loss: 0.0197
Epoch 8/10
40/40 [==============================] - 346s 9s/step - loss: 0.0209 - val_loss: 0.0197
Epoch 9/10
40/40 [==============================] - 341s 9s/step - loss: 0.0200 - val_loss: 0.0187
Epoch 10/10
5/5 [==============================] - 20s 4s/step


In [ ]:
reconstructed_images = autoencoder.predict(test_images)

5/5 [==============================] - 27s 5s/step


In [ ]:
import numpy as np
from skimage.metrics import mean_squared_error, peak_signal_noise_ratio, structural_similarity

# Flatten the images for MSE calculation
reconstructed_images_flat = reconstructed_images.reshape(reconstructed_images.shape[0], -1)
test_images_flat = test_images.reshape(test_images.shape[0], -1)

# Calculate MSE
mse = mean_squared_error(test_images_flat, reconstructed_images_flat)
print("Mean Squared Error (MSE):", mse)

# Calculate PSNR
psnr = peak_signal_noise_ratio(test_images_flat, reconstructed_images_flat)
print("Peak Signal-to-Noise Ratio (PSNR):", psnr)

# Calculate SSIM
ssim = structural_similarity(test_images, reconstructed_images, multichannel=True)
print("Structural Similarity Index (SSIM):", ssim)


Mean Squared Error (MSE): 0.019570083992108946
Peak Signal-to-Noise Ratio (PSNR): 17.084073104058128


<ipython-input-6-0f001efe9259>:17: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim = structural_similarity(test_images, reconstructed_images, multichannel=True)


Structural Similarity Index (SSIM): 0.8191226


In [ ]:
import numpy as np
from skimage.metrics import peak_signal_noise_ratio

# Calculate PSNR scores for each image
psnr_scores = []
for i in range(len(test_images)):
    orig_image = np.clip(test_images[i] * 255., 0, 255).astype(np.uint8)
    recon_image = np.clip(reconstructed_images[i] * 255., 0, 255).astype(np.uint8)
    psnr_val = peak_signal_noise_ratio(orig_image, recon_image)
    psnr_scores.append(psnr_val)

# Convert PSNR scores to a numpy array
psnr_scores = np.array(psnr_scores)

# Calculate maximum and minimum PSNR values
max_psnr = np.max(psnr_scores)
min_psnr = np.min(psnr_scores)

print("Maximum PSNR:", max_psnr)
print("Minimum PSNR:", min_psnr)


Maximum PSNR: 23.926624935952084
Minimum PSNR: 10.046588833982803


TRIAL 2


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt

# Set the path to the downloaded DIV2K dataset folder
dataset_folder = "/content/datasets/DIV2K_train_HR"  # Replace with the actual path

# Set the desired image size
image_size = (256, 256)  # Replace with your desired image size

# Load and preprocess the dataset
def load_dataset():
    images = []
    img_list = os.listdir(dataset_folder)
    for img_name in img_list:
        img_path = os.path.join(dataset_folder, img_name)
        img = load_img(img_path, target_size=image_size)
        img = img_to_array(img) / 255.0  # Normalize pixel values
        images.append(img)
    images = np.array(images)
    return images

# Split dataset into training and testing sets
images = load_dataset()
train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

# Define the compression model
input_img = Input(shape=image_size + (3,))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded_img = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Create the model
autoencoder = Model(input_img, decoded_img)
autoencoder.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model
autoencoder.fit(train_images, train_images, epochs=30, batch_size=256, validation_data=(test_images, test_images))

# Reconstruct images
reconstructed_images = autoencoder.predict(test_images)

Epoch 1/30
3/3 [==============================] - 31s 5s/step - loss: 0.0846 - val_loss: 0.0750
Epoch 2/30
3/3 [==============================] - 2s 685ms/step - loss: 0.0783 - val_loss: 0.0698
Epoch 3/30
3/3 [==============================] - 2s 770ms/step - loss: 0.0723 - val_loss: 0.0635
Epoch 4/30
3/3 [==============================] - 2s 673ms/step - loss: 0.0651 - val_loss: 0.0561
Epoch 5/30
3/3 [==============================] - 2s 690ms/step - loss: 0.0569 - val_loss: 0.0478
Epoch 6/30
3/3 [==============================] - 2s 708ms/step - loss: 0.0479 - val_loss: 0.0401
Epoch 7/30
3/3 [==============================] - 2s 693ms/step - loss: 0.0397 - val_loss: 0.0338
Epoch 8/30
3/3 [==============================] - 2s 726ms/step - loss: 0.0335 - val_loss: 0.0294
Epoch 9/30
3/3 [==============================] - 2s 738ms/step - loss: 0.0290 - val_loss: 0.0256
Epoch 10/30
3/3 [==============================] - 2s 711ms/step - loss: 0.0251 - val_loss: 0.0222
Epoch 11/30
3/3 [====

In [ ]:
import numpy as np
from skimage.metrics import peak_signal_noise_ratio

# Calculate PSNR scores for each image
psnr_scores = []
for i in range(len(test_images)):
    orig_image = np.clip(test_images[i] * 255., 0, 255).astype(np.uint8)
    recon_image = np.clip(reconstructed_images[i] * 255., 0, 255).astype(np.uint8)
    psnr_val = peak_signal_noise_ratio(orig_image, recon_image)
    psnr_scores.append(psnr_val)

# Convert PSNR scores to a numpy array
psnr_scores = np.array(psnr_scores)

# Calculate maximum and minimum PSNR values
max_psnr = np.max(psnr_scores)
min_psnr = np.min(psnr_scores)

print("Maximum PSNR:", max_psnr)
print("Minimum PSNR:", min_psnr)


Maximum PSNR: 28.248002684767357
Minimum PSNR: 15.08252340014867
